In [1]:
import math

import numpy as np
import pandas as pd

# Load file with clustered data

In [2]:
df_clustered = pd.read_csv("download_imgs/clustered_points.csv")
df_clustered

/var/folders/kw/9lnn_1r159n28p1krywmf8_w0000gq/T/ipykernel_35043/2871462531.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clustered = pd.read_csv("download_imgs/clustered_points.csv")


,Unnamed: 0,Unnamed: 0_x,uuid_x,source,orig_id,city,city_ascii,city_id,city_lat,city_lon,...,Car-Mount,Ego-Vehicle,Total,visual_complexity,Road_ratio,Vegetation_ratio,Car_ratio,Building_ratio,Sky_ratio,cluster_label
0,0,37185,1c80da17-697e-485b-b513-7925b07d8d92,Mapillary,865783177614020,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,2044260.0,1.876654,0.118965,0.060797,0.016989,0.136482,0.278301,0
1,1,37221,e256fdf0-c55f-48fc-b55c-13b310243e3b,Mapillary,826518501302716,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,107425.0,2005395.0,1.993278,0.079747,0.248659,0.005505,0.037918,0.323582,0
2,2,37223,8fae8f17-37bc-4316-bc1a-49f5e932e818,Mapillary,285162226565982,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,3043680.0,1.326514,0.141799,0.607542,0.013955,0.000000,0.128782,1
3,3,37229,aefe21c3-f761-4a7f-b332-cccef585f556,Mapillary,300351374915837,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,3101724.0,1.837808,0.098427,0.391328,0.025908,0.085638,0.248062,0
4,4,37237,2b21fe5c-7dff-47ee-a9d7-de29e60c6db0,Mapillary,240760857851346,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,1991090.0,1.568897,0.111652,0.270545,0.000000,0.009440,0.368394,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31449,31449,37148,58f7ab3a-95f2-4e63-aa52-9111798c12c9,Mapillary,1188476305232671,Santiago,Santiago,1152554349,-33.45,-70.6667,...,46230.0,189570.0,2244579.0,2.128048,0.291580,0.122134,0.081733,0.089744,0.175526,1
31450,31450,37151,eab7267a-84fa-48b3-a71b-3f98ff0b590f,Mapillary,590854812050122,Santiago,Santiago,1152554349,-33.45,-70.6667,...,0.0,153800.0,3056084.0,1.384411,0.112094,0.012597,0.070912,0.078220,0.620889,3
31451,31451,37153,6c330b03-2960-4456-9683-33c2414a2f40,Mapillary,317767673096061,Santiago,Santiago,1152554349,-33.45,-70.6667,...,0.0,31035.0,1968085.0,2.309947,0.107196,0.149529,0.013254,0.154942,0.218123,2
31452,31452,37156,1f3e335f-9029-49f2-8e4a-6b5f522f5bbf,Mapillary,486233076032026,Santiago,Santiago,1152554349,-33.45,-70.6667,...,0.0,128150.0,2001520.0,1.505885,0.001237,0.142749,0.000452,0.384116,0.043040,1


In [3]:
df_clustered.groupby("city").size()

city
Abuja              178
Amsterdam         3443
San Francisco    25349
Santiago           629
Singapore         1855
dtype: int64

# Random sample

In [4]:
# for each city, randomly sample a total of 100 images, evenly distributed from each cluster
city_id = [1702341327, 1528355309, 1840021543, 1566342270, 1152554349]

city_names = {
    1702341327: "Singapore",
    1528355309: "Amsterdam",
    1840021543: "San Francisco",
    1566342270: "Abuja",
    1152554349: "Santiago",
}


df_samples = pd.DataFrame(columns=df_clustered.columns)

In [5]:
NUM_SVIS = 100

In [6]:
df_clustered.groupby("city").size()

city
Abuja              178
Amsterdam         3443
San Francisco    25349
Santiago           629
Singapore         1855
dtype: int64

In [7]:
# keep track of the samples
df_sampled = pd.DataFrame()

# keep track of already sampled indices
sampled_indices = []

# iterate over each city
for id in city_id:

    # DEBUG
    if id == 1566342270: # skip Abuja until the new images are downloaded.
        continue
    # DEBUG
    
    print(f"City: {city_names[id]}")
    df_city = df_clustered[df_clustered["city_id"] == id]

    # count the number of rows in each cluster
    cluster_counts = df_city["cluster_label"].value_counts()

    # calculate the number of samples to take from each cluster
    samples_per_cluster = np.floor_divide(NUM_SVIS, len(cluster_counts))

    # keep track of the samples
    df_sampled_city = pd.DataFrame()

    # sample from each cluster
    for cluster_label, count in cluster_counts.items():
        if count <= samples_per_cluster:
            # if the cluster has fewer samples than required, take all samples
            df_sampled_city = pd.concat(
                [df_sampled_city, df_city[df_city["cluster_label"] == cluster_label]]
            )
        else:
            # sample evenly from the cluster
            available_samples = df_city[df_city["cluster_label"] == cluster_label].index.difference(sampled_indices)
            if len(available_samples) == 0:
                continue  # Skip this cluster if no available samples
            sampled_from_cluster = np.random.choice(available_samples, size=samples_per_cluster, replace=False)
            sampled_indices.extend(sampled_from_cluster)
            df_sampled_city = pd.concat(
                [
                    df_sampled_city,
                    df_city.loc[sampled_from_cluster],
                ]
            )

    # if the total number of samples taken so far is less than 100, take additional samples
    remaining_samples = NUM_SVIS - len(df_sampled_city)

    if remaining_samples > 0:
        print(f"There are still {remaining_samples} sample(s) required")
        for cluster_label, count in cluster_counts.items():
            if count > samples_per_cluster:
                # sample from clusters with more samples
                available_samples = df_city[df_city["cluster_label"] == cluster_label].index.difference(sampled_indices)
                if len(available_samples) == 0:
                    continue  # Skip this cluster if no available samples
                additional_samples = min(remaining_samples, len(available_samples))
                new_samples = np.random.choice(available_samples, size=additional_samples, replace=False)

                sampled_indices.extend(new_samples)
                df_sampled_city = pd.concat(
                    [
                        df_sampled_city,
                        df_city.loc[new_samples],
                    ]
                )
                remaining_samples -= additional_samples
                if remaining_samples == 0:
                    break

    # ensure that the final sampled DataFrame has exactly NUM_SVIS rows
    df_sampled_city = df_sampled_city.sample(n=NUM_SVIS, random_state=42)

    # reset index
    df_sampled_city.reset_index(drop=True, inplace=True)

    # concatenate all cities
    df_sampled = pd.concat([df_sampled, df_sampled_city], ignore_index=True)


City: Singapore
City: Amsterdam
City: San Francisco
City: Santiago


In [8]:
df_sampled

,Unnamed: 0,Unnamed: 0_x,uuid_x,source,orig_id,city,city_ascii,city_id,city_lat,city_lon,...,Car-Mount,Ego-Vehicle,Total,visual_complexity,Road_ratio,Vegetation_ratio,Car_ratio,Building_ratio,Sky_ratio,cluster_label
0,1318,55503,d8a65020-f9ae-4d0c-81c2-7dbd5a6e9a2b,KartaView,393391790,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,5187208.0,1.769936,0.106477,0.185643,0.006937,0.430934,0.126584,3
1,1750,59107,9ca21876-0c28-47a6-87bd-6973507aa5ca,KartaView,1386320153,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,4817770.0,1.547210,0.277635,0.094870,0.000000,0.001999,0.004782,2
2,1306,55440,47497ae8-899e-4fbb-9498-2a65c8e1d252,KartaView,393462414,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,5167272.0,1.910432,0.076478,0.325374,0.101395,0.027987,0.148759,4
3,254,40819,8e5cd826-fb2c-4085-b271-4c2ef1c7e5cf,Mapillary,2908647492737508,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,2296977.0,1.580716,0.436126,0.276187,0.002696,0.000643,0.005007,2
4,189,39767,06a40aca-e758-499e-8d73-9f755099c06b,Mapillary,977516106319840,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,0.0,2030725.0,1.422093,0.340484,0.489731,0.012340,0.019284,0.027995,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,31197,32783,960e05de-47a0-40cd-90d0-3665e0fdff57,Mapillary,407558607379829,Santiago,Santiago,1152554349,-33.45,-70.6667,...,0.0,145528.0,3010324.0,1.415317,0.128966,0.060541,0.037033,0.044802,0.624189,3
396,31171,32074,342b1d4c-a247-4536-b216-4d605bfa05fb,Mapillary,143793214634914,Santiago,Santiago,1152554349,-33.45,-70.6667,...,0.0,10632.0,3065620.0,1.689184,0.119783,0.008276,0.001966,0.239543,0.448953,3
397,31447,37137,7029069e-d70a-4a47-bd0e-0aad3ae3feb4,Mapillary,4168618643188204,Santiago,Santiago,1152554349,-33.45,-70.6667,...,0.0,585.0,2015635.0,1.800189,0.331308,0.180878,0.009126,0.242990,0.115785,1
398,31187,32399,1c1b57b6-8c74-4b77-86cb-97d875573375,Mapillary,793584458183930,Santiago,Santiago,1152554349,-33.45,-70.6667,...,0.0,0.0,2009815.0,1.944649,0.271796,0.054027,0.184870,0.142722,0.232788,0


# Save to csv

In [9]:
df_sampled["uuid"] = df_sampled["uuid_x"]
df_sampled.to_csv("download_imgs/filtered_points.csv")
df_sampled

,Unnamed: 0,Unnamed: 0_x,uuid_x,source,orig_id,city,city_ascii,city_id,city_lat,city_lon,...,Ego-Vehicle,Total,visual_complexity,Road_ratio,Vegetation_ratio,Car_ratio,Building_ratio,Sky_ratio,cluster_label,uuid
0,1318,55503,d8a65020-f9ae-4d0c-81c2-7dbd5a6e9a2b,KartaView,393391790,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,5187208.0,1.769936,0.106477,0.185643,0.006937,0.430934,0.126584,3,d8a65020-f9ae-4d0c-81c2-7dbd5a6e9a2b
1,1750,59107,9ca21876-0c28-47a6-87bd-6973507aa5ca,KartaView,1386320153,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,4817770.0,1.547210,0.277635,0.094870,0.000000,0.001999,0.004782,2,9ca21876-0c28-47a6-87bd-6973507aa5ca
2,1306,55440,47497ae8-899e-4fbb-9498-2a65c8e1d252,KartaView,393462414,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,5167272.0,1.910432,0.076478,0.325374,0.101395,0.027987,0.148759,4,47497ae8-899e-4fbb-9498-2a65c8e1d252
3,254,40819,8e5cd826-fb2c-4085-b271-4c2ef1c7e5cf,Mapillary,2908647492737508,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,2296977.0,1.580716,0.436126,0.276187,0.002696,0.000643,0.005007,2,8e5cd826-fb2c-4085-b271-4c2ef1c7e5cf
4,189,39767,06a40aca-e758-499e-8d73-9f755099c06b,Mapillary,977516106319840,Singapore,Singapore,1702341327,1.30,103.8000,...,0.0,2030725.0,1.422093,0.340484,0.489731,0.012340,0.019284,0.027995,2,06a40aca-e758-499e-8d73-9f755099c06b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,31197,32783,960e05de-47a0-40cd-90d0-3665e0fdff57,Mapillary,407558607379829,Santiago,Santiago,1152554349,-33.45,-70.6667,...,145528.0,3010324.0,1.415317,0.128966,0.060541,0.037033,0.044802,0.624189,3,960e05de-47a0-40cd-90d0-3665e0fdff57
396,31171,32074,342b1d4c-a247-4536-b216-4d605bfa05fb,Mapillary,143793214634914,Santiago,Santiago,1152554349,-33.45,-70.6667,...,10632.0,3065620.0,1.689184,0.119783,0.008276,0.001966,0.239543,0.448953,3,342b1d4c-a247-4536-b216-4d605bfa05fb
397,31447,37137,7029069e-d70a-4a47-bd0e-0aad3ae3feb4,Mapillary,4168618643188204,Santiago,Santiago,1152554349,-33.45,-70.6667,...,585.0,2015635.0,1.800189,0.331308,0.180878,0.009126,0.242990,0.115785,1,7029069e-d70a-4a47-bd0e-0aad3ae3feb4
398,31187,32399,1c1b57b6-8c74-4b77-86cb-97d875573375,Mapillary,793584458183930,Santiago,Santiago,1152554349,-33.45,-70.6667,...,0.0,2009815.0,1.944649,0.271796,0.054027,0.184870,0.142722,0.232788,0,1c1b57b6-8c74-4b77-86cb-97d875573375


In [10]:
df_simple_metadata = df_sampled[
    ["source", "orig_id", "city", "country", "continent", "uuid"]
]
df_simple_metadata.to_csv("download_imgs/metadata_points.csv")
df_simple_metadata

,source,orig_id,city,country,continent,uuid
0,KartaView,393391790,Singapore,Singapore,Asia,d8a65020-f9ae-4d0c-81c2-7dbd5a6e9a2b
1,KartaView,1386320153,Singapore,Singapore,Asia,9ca21876-0c28-47a6-87bd-6973507aa5ca
2,KartaView,393462414,Singapore,Singapore,Asia,47497ae8-899e-4fbb-9498-2a65c8e1d252
3,Mapillary,2908647492737508,Singapore,Singapore,Asia,8e5cd826-fb2c-4085-b271-4c2ef1c7e5cf
4,Mapillary,977516106319840,Singapore,Singapore,Asia,06a40aca-e758-499e-8d73-9f755099c06b
...,...,...,...,...,...,...
395,Mapillary,407558607379829,Santiago,Chile,South America,960e05de-47a0-40cd-90d0-3665e0fdff57
396,Mapillary,143793214634914,Santiago,Chile,South America,342b1d4c-a247-4536-b216-4d605bfa05fb
397,Mapillary,4168618643188204,Santiago,Chile,South America,7029069e-d70a-4a47-bd0e-0aad3ae3feb4
398,Mapillary,793584458183930,Santiago,Chile,South America,1c1b57b6-8c74-4b77-86cb-97d875573375


In [11]:
df_sampled["uuid"].nunique()

400

In [12]:
df_sampled.groupby("city").size()

city
Amsterdam        100
San Francisco    100
Santiago         100
Singapore        100
dtype: int64